<h1>Assignment 1</h1>

<h2>Loading the Data</h2>
<p>This code snippet is the one provedid to us in "starting_script.py".</p>

In [6]:
pip install sklearn

Note: you may need to restart the kernel to use updated packages.


In [4]:
import numpy as np

DATA_INDEX = 0 # Can be 0,1,2,3,4
file = f"./data/dataset-{DATA_INDEX}.npy" # this f expression requires Python 3
data = np.load(file) # loads the file
X, Y = data[:,:-1], data[:,-1] # last column represents targets Y, the other columns the input features

In [4]:
print(X)

[[-3.30945353 -4.59571161 -0.25663391 ...  4.85446625 -2.54075666
   1.55370229]
 [ 5.72554978 -0.25120372  3.34201656 ...  0.92833202  3.26187419
  -0.40702635]
 [-1.9921959  -1.17021443  1.08655897 ...  3.44695246 -2.52676775
   0.68051378]
 ...
 [-0.68264084 -2.41163357  0.28513491 ... -3.8254249   2.40653586
  -6.50226474]
 [-4.05556948  1.44448798  2.79663958 ... -0.93270131 -6.47108348
  -1.62041453]
 [ 4.33054323  0.2750504   5.15343234 ...  1.44232185 -0.96185653
   1.9774413 ]]


In [5]:
print(Y)

[8. 9. 9. 5. 5. 6. 4. 0. 4. 5. 9. 8. 8. 7. 0. 9. 9. 8. 4. 6. 4. 9. 4. 0.
 4. 1. 3. 8. 3. 2. 8. 4. 9. 1. 9. 7. 5. 5. 0. 4. 9. 5. 3. 9. 8. 8. 1. 7.
 1. 1. 2. 6. 4. 5. 2. 8. 3. 5. 2. 4. 6. 2. 8. 4. 1. 0. 7. 2. 7. 3. 8. 0.
 7. 8. 2. 1. 3. 2. 5. 7. 1. 3. 6. 8. 7. 0. 5. 6. 7. 7. 4. 9. 6. 7. 5. 5.
 0. 0. 2. 4. 7. 7. 1. 5. 2. 0. 4. 7. 3. 8. 1. 0. 5. 0. 1. 4. 5. 7. 4. 5.
 3. 4. 3. 1. 8. 6. 7. 1. 2. 6. 9. 9. 6. 3. 3. 6. 9. 6. 5. 4. 4. 0. 2. 3.
 5. 2. 9. 9. 3. 4. 8. 2. 7. 8. 0. 6. 9. 0. 7. 6. 8. 1. 6. 0. 1. 9. 2. 3.
 2. 1. 7. 6. 0. 0. 0. 1. 9. 9. 8. 5. 2. 2. 3. 6. 1. 1. 7. 6. 1. 3. 7. 3.
 8. 6. 2. 3. 5. 0. 5. 3.]


<p>Instead of changing the DATA_INDEX variable before rerunning this Jupyter notebook for each separate dataset, I have decided to write a function based ont the snipped above, which will automate the process somewhat more.</p>

In [2]:
# A vector containing the dataset indexes
data_indexes = {0, 1, 2, 3, 4}


# A function that loads a dataset when given its index
def load_data(data_index):
    file = f"./data/dataset-{data_index}.npy" # this f expression requires Python 3
    data = np.load(file) # loads the file
    X, Y = data[:,:-1], data[:,-1] # last column represents targets Y, the other columns the input features
    
    return X, Y

In [5]:
# Evaluating the size of the datasets
for data_index in data_indexes:
    X, y = load_data(data_index)
    print(data_index, len(X))

0 200
1 800
2 1500
3 3000
4 5000


<h2>Splitting the Dataset into Train, Validation and Test Sets</h2>
<p>The training, validation and test sets will contain 70%, 20%, and 10% of the data, respectively.</p>
<p>https://datascience.stackexchange.com/questions/15135/train-test-validation-set-splitting-in-sklearn</p>

In [7]:
from sklearn.model_selection import train_test_split


def split_data(X,Y):
    # Split the dataset into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=0)
    # Then, split the train set once again into separate train and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=2/9, random_state=0) # 2/9 x 0.9 = 0.2
    
    return X_train, X_val, X_test, y_train, y_val, y_test

<h2>Random Seed</h2>
<p>Before starting to properly work on solving this assignment, I will set the random seed to an arbitrary value, in order to ensure the reproducibility of my results.</p>

In [8]:
import random

random.seed(43)

<h2>A. Grid Search</h2>
The first approach I will use for hyperparameter optimization is Grid Search. First, I will select discrete values for each of the hyperparameters that interest me, then I will iterate through all of their combinations and maintain only the one resulting in the highest accuracy.

<h3>Step 1: Defining Discrete Values for the Hyperparameters</h3>
Here is an attempt at establishing which hyperparameters I should optimize and what discrete values I should consider for them. The parameters of the sklearn.svm.SVC class are the following:
<ul>
    <li><b>C</b> - the degree of regularization, values: {0.01, 0.1, 1, 10, 100, 1000}</li>
    <li><b>kernel</b> - values: {‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’}</li>
    <li><b>degree</b> - only for 'poly' kernel, values: {0, 1, 2, 3, 4, 5, 6}</li>
    <li><b>gamma</b> - kernel coefficient for ‘rbf’, ‘poly’ and ‘sigmoid’, values: {'scale', 'auto', 0.1, 1, 10, 100]</li>
    <li><b>coef0</b> - only significant in ‘poly’ and ‘sigmoid’, values: {0.01, 0.1, 1, 10, 100, 1000}</li>
    <li><b>shrinking - values: {True, False}</b></li>
    <li><b>probability - values: {True, False}</b></li>
    <li><b>tol</b> - values: {1e-5, 1e-4, 1e-3, 1e-2, 1e-1}</li>
    <li><b>cache_size</b> - values: {100, 200, 300, 400, 500}</li>
    <li><b>class_weight</b> - will not optimize</li>
    <li><b>verbose</b> - should not be optimized</li>
    <li><b>max_iter</b> - will not optimize</li>
    <li><b>decision_function_shape</b> - should not be optimized</li>
    <li><b>break_ties</b> - values: {True, False}</li>
    <li><b>random_state</b> - should not be optimized, will choose 10 each time</li>
</ul>
 More details at: https://medium.com/all-things-ai/in-depth-parameter-tuning-for-svc-758215394769.

<h3>Step 2: Performing and Evaluating Grid Search</h3>

In [13]:
from sklearn import svm
from sklearn.metrics import accuracy_score


# A not so important function that will help print the results
def pretty_print(max_C, max_kernel, max_degree, max_gamma, test_accuracy, data_index):
    print("----------------------------------------------------------------------")
    print("For the dataset with index {0}, the accuracy on the test set is {1}.".format(data_index, test_accuracy))
    print("Hyperparameters of the SVC predictor:")
    print("C = {0}".format(max_C))
    print("kernel = {0}".format(max_kernel))
    
    # Degree only matters for 'poly' kernels
    if max_kernel == 'poly':
        print("degree = {0}".format(max_degree))
        
    # Gamma only matters for 'poly', 'rbf', and 'sigmoid' kernels
    if max_kernel in {'poly', 'rbf', 'sigmoid'}:
        print("gamma = {0}".format(max_gamma))
        
    print("----------------------------------------------------------------------")


        
def grid_search(data_index):
    #Load the dataset
    X,Y = load_data(data_index)
    #print("Data Y ", Y)
    
    # Split the data into training, validation and test sets
    X_train, X_val, X_test, y_train, y_val, y_test = split_data(X, Y)
    
    # Save the highest accuracy in a variable
    max_accuracy = 0
    #print("Max accuracy ", max_accuracy)
    
    # Iterate through all the parameters' values
    for C in [0.01, 0.1, 1, 10, 100, 1000]:
        for kernel in ['linear', 'poly', 'rbf', 'sigmoid']:
            
            if kernel == 'linear':
                # Create and fit the model
                svc = svm.SVC(C=C, kernel=kernel)
                svc.fit(X_train, y_train)

                # Evaluate the model's accuracy on the validation set
                y_val_pred = svc.predict(X_val)
                accuracy = accuracy_score(y_val, y_val_pred)
                
                if accuracy >= max_accuracy:
                    max_accuracy = accuracy
                    max_C = C
                    max_kernel = kernel
                
            else:
                for gamma in  ['scale', 'auto']:
                    if kernel == 'poly':
                        for degree in [1, 3, 5, 7, 10, 15, 20]:
                            # Create and fit the model
                            svc = svm.SVC(C=C, kernel=kernel, degree=degree, gamma=gamma)
                            svc.fit(X_train, y_train)

                            # Evaluate the model's accuracy on the validation set
                            y_val_pred = svc.predict(X_val)
                            accuracy = accuracy_score(y_val, y_val_pred)
                            
                            if accuracy >= max_accuracy:
                                max_accuracy = accuracy
                                max_C = C
                                max_kernel = kernel
                                max_gamma = gamma
                                max_degree = degree
                                
                    else:
                        # Create and fit the model
                        svc = svm.SVC(C=C, kernel=kernel, gamma=gamma)
                        svc.fit(X_train, y_train)

                        # Evaluate the model's accuracy on the validation set
                        y_val_pred = svc.predict(X_val)
                        accuracy = accuracy_score(y_val, y_val_pred)
                        
                        if accuracy >= max_accuracy:
                            max_accuracy = accuracy
                            max_C = C
                            max_kernel = kernel
                            max_gamma = gamma
    
    # At the end, recreate and fit one last time the SVC that had the best accuracy
    #svc_best = svm.SVC(C=max_C, kernel=max_kernel, degree=max_degree, gamma=max_gamma)
    svc_best = svm.SVC(C=max_C, kernel=max_kernel, degree=max_degree, gamma=max_gamma)
    svc_best.fit(X_train, y_train)
    
    # Now, evaluate the accuracy on the test set
    y_test_pred = svc_best.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    
    pretty_print(max_C, max_kernel, max_degree, max_gamma, test_accuracy, data_index)
    
    del svc_best

In [108]:
# Finally, we can see the results
for data_index in data_indexes:
    grid_search(data_index)

----------------------------------------------------------------------
For the dataset with index 0, the accuracy on the test set is 0.7.
Hyperparameters of the SVC predictor:
C = 1000
kernel = rbf
gamma = scale
----------------------------------------------------------------------
----------------------------------------------------------------------
For the dataset with index 1, the accuracy on the test set is 0.725.
Hyperparameters of the SVC predictor:
C = 1000
kernel = rbf
gamma = scale
----------------------------------------------------------------------
----------------------------------------------------------------------
For the dataset with index 2, the accuracy on the test set is 0.7066666666666667.
Hyperparameters of the SVC predictor:
C = 1000
kernel = rbf
gamma = scale
----------------------------------------------------------------------
----------------------------------------------------------------------
For the dataset with index 3, the accuracy on the test set is 0

<h2>B. Random Search</h2>
The second approach I will use for hyperparameter optimization is Random Search. First, I will select ranges for each of the hyperparameters that interest me, then I will iterate through all of their combinations and maintain only the one resulting in the highest accuracy. I will maintain the same number of combinations for hyperparameters as for the grid search.

In [58]:
def random_search(data_index):
    #Load the dataset
    X,Y = load_data(data_index)
    #print("Data Y ", Y)
    
    # Split the data into training, validation and test sets
    X_train, X_val, X_test, y_train, y_val, y_test = split_data(X, Y)
    
    # Save the highest accuracy in a variable
    max_accuracy = 0 # starts as 0, is updated when a higher accuracy is found
    max_C = 0
    max_kernel = 0
    max_degree = 0
    max_gamma = 0
    #print("Max accuracy ", max_accuracy)
    
    # Iterate through all the parameters' values
    for i in range(102):
        C = random.uniform(0.01, 1000)
        kernel = random.choice(['linear', 'poly', 'rbf', 'sigmoid'])
        if kernel == 'linear':
            # Create and fit the model
            svc = SVC(C=C, kernel=kernel)
            svc.fit(X_train, y_train)

            # Evaluate the model's accuracy on the validation set
            y_val_pred = svc.predict(X_val)
            accuracy = accuracy_score(y_val, y_val_pred)
                
            # If the accuracy is higher than the current maximum,
            # then update the maximum accuracy and save the values of 
            # the hyperparameters that led to it.
            if accuracy > max_accuracy:
                max_accuracy = accuracy
                max_C = C
                max_kernel = kernel
                
            #print(svc, accuracy)
                
            del svc
            
        else:
            gamma = random.choice(['scale', 'auto'])
            if kernel == 'poly':
                degree = random.randint(1,20)
                # Create and fit the model
                svc = SVC(C=C, kernel=kernel, degree=degree, gamma=gamma)
                svc.fit(X_train, y_train)

                # Evaluate the model's accuracy on the validation set
                y_val_pred = svc.predict(X_val)
                accuracy = accuracy_score(y_val, y_val_pred)
                            
                if accuracy > max_accuracy:
                    max_accuracy = accuracy
                    max_C = C
                    max_kernel = kernel
                    max_degree = degree
                    max_gamma = gamma
                    
                #print(svc, accuracy)
                
                del svc
                
            else:
                # Create and fit the model
                svc = SVC(C=C, kernel=kernel, gamma=gamma)
                svc.fit(X_train, y_train)

                # Evaluate the model's accuracy on the validation set
                y_val_pred = svc.predict(X_val)
                accuracy = accuracy_score(y_val, y_val_pred)
                        
                if accuracy >= max_accuracy:
                    max_accuracy = accuracy
                    max_C = C
                    max_kernel = kernel
                    max_gamma = gamma
                    
                #print(svc, accuracy)
                
                del svc
    
    # At the end, recreate and fit one last time the SVC that had the best accuracy
    svc_best = SVC(C=max_C, kernel=max_kernel, degree=max_degree, gamma=max_gamma)
    svc_best.fit(X_train, y_train)
    
    # Now, evaluate the accuracy on the test set
    y_test_pred = svc_best.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    
    pretty_print(max_C, max_kernel, max_degree, max_gamma, test_accuracy, data_index)
    
    del svc_best

In [59]:
# Check out the results
for data_index in data_indexes:
    random_search(data_index)

----------------------------------------------------------------------
For the dataset with index 0, the accuracy on the test set is 0.7.
Hyperparameters of the SVC predictor:
C = 327.15468430788815
kernel = rbf
gamma = scale
----------------------------------------------------------------------
----------------------------------------------------------------------
For the dataset with index 1, the accuracy on the test set is 0.725.
Hyperparameters of the SVC predictor:
C = 346.29691310600754
kernel = rbf
gamma = scale
----------------------------------------------------------------------
----------------------------------------------------------------------
For the dataset with index 2, the accuracy on the test set is 0.96.
Hyperparameters of the SVC predictor:
C = 447.8930716717264
kernel = poly
degree = 2
gamma = auto
----------------------------------------------------------------------
----------------------------------------------------------------------
For the dataset with inde

<h2>C. Bayesian Optimization</h2>

An unfinished attempt.

In [107]:
import numpy as np
from scipy.stats import norm
from scipy.optimize import minimize
from sklearn.gaussian_process import GaussianProcessRegressor
#from sklearn.gaussian_process.kernels import ConstantKernel, Matern


# Get a random pair of values from X and y
def query(X, y):
    index = random.randint(0, len(X))
    return X[index], y[index]


# Aquisition function
def expected_improvement(f_star, mu, sigma):
    z = (f_star - mu) / sigma
    return (f_star - mu) * norm.cdf(z) + sigma * norm.pdf(z)


# Decide where to sample next
def next_x(X_known, y_known, gpr):
    dims = len(X_known[0])
    bounds = []
    for i in range(dims):
        bounds.append((-10, 10))
        
    gpr.fit(X_known, y_known)
    
    y_min = 1
    arg_x_min = None
    
    def min_objective(X):
        X = X.reshape(-1, dims)
        f_star, sigma = gpr.predict(X, return_std=True)
        sigma = sigma.reshape(-1, 1)
        mu = gpr.predict(X_known)
        val = -expected_improvement(f_star, mu, sigma).sum()
        #print(val)
        return val
    
    # Search starting from 20 different points, to avoid landing in a local minimum
    rand_x = np.random.uniform(-10, 10, size=(10, dims))
    for i in rand_x:
        result = minimize(min_objective, x0=i, bounds=bounds, method='L-BFGS-B')
        
        # Find the minimum of the aquisition function
        if result.fun <= y_min:
            y_min = result.fun
            print(X)
            arg_x_min = np.where(X == result.x)
    
    # Return the x vale corresponding to the minimum
    return arg_x_min

# load dataset
X, y = load_data(0)

# Create sets of known data points and add two points for now
X_known, y_known = [], []
xi, yi = query(X, y)
X_known.append(xi)
y_known.append(yi)
xi, yi = query(X, y)
X_known.append(xi)
y_known.append(yi)

gpr = GaussianProcessRegressor()

for i in range(10):
    gpr.fit(X_known, y_known)
    next_x(X_known, y_known, gpr)
    print(arg_x_next[0])
    x_next = X[arg_x_next]
    y_next = y[arg_x_next]
    
    X_known.append(x_next)
    y_known.append(y_next)

[[-3.30945353 -4.59571161 -0.25663391 ...  4.85446625 -2.54075666
   1.55370229]
 [ 5.72554978 -0.25120372  3.34201656 ...  0.92833202  3.26187419
  -0.40702635]
 [-1.9921959  -1.17021443  1.08655897 ...  3.44695246 -2.52676775
   0.68051378]
 ...
 [-0.68264084 -2.41163357  0.28513491 ... -3.8254249   2.40653586
  -6.50226474]
 [-4.05556948  1.44448798  2.79663958 ... -0.93270131 -6.47108348
  -1.62041453]
 [ 4.33054323  0.2750504   5.15343234 ...  1.44232185 -0.96185653
   1.9774413 ]]
[[-3.30945353 -4.59571161 -0.25663391 ...  4.85446625 -2.54075666
   1.55370229]
 [ 5.72554978 -0.25120372  3.34201656 ...  0.92833202  3.26187419
  -0.40702635]
 [-1.9921959  -1.17021443  1.08655897 ...  3.44695246 -2.52676775
   0.68051378]
 ...
 [-0.68264084 -2.41163357  0.28513491 ... -3.8254249   2.40653586
  -6.50226474]
 [-4.05556948  1.44448798  2.79663958 ... -0.93270131 -6.47108348
  -1.62041453]
 [ 4.33054323  0.2750504   5.15343234 ...  1.44232185 -0.96185653
   1.9774413 ]]
[[-3.30945353 -4

IndexError: too many indices for array